In [129]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv as csv

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier

In [130]:
train_df = pd.read_csv('C:/Users/Adarsh Saavarn/Desktop/titanic/train.csv')
test_df = pd.read_csv('C:/Users/Adarsh Saavarn/Desktop/titanic/test.csv')

In [131]:
np.random.seed(0)

In [132]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [133]:
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [134]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [135]:

test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [136]:
#Dropping some columns which is useless in prediction
train_df=train_df.drop(['PassengerId','Name','Ticket','Cabin'],axis=1)
test_df=test_df.drop(['Name','Ticket','Cabin'],axis=1)

In [137]:
#replacing Sex to int mapping
train_df['Sex'] = train_df['Sex'].map({'male':1,'female':0}).astype(int)
test_df['Sex'] = test_df['Sex'].map({'male':1,'female':0}).astype(int)

In [138]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null int32
Age         714 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
Embarked    889 non-null object
dtypes: float64(2), int32(1), int64(4), object(1)
memory usage: 52.3+ KB


In [139]:
print(train_df.describe(include=['object']))
train_df['Embarked'] = train_df['Embarked'].fillna('S')
print('-'*30)
print(train_df.describe(include=['object']))

train_df['Embarked'] = train_df['Embarked'].map({'C':0,'Q':1,'S':2}).astype(int)
test_df['Embarked']  = test_df['Embarked'].map({'C':0,'Q':1,'S':2}).astype(int)

       Embarked
count       889
unique        3
top           S
freq        644
------------------------------
       Embarked
count       891
unique        3
top           S
freq        646


In [140]:
mean_age = train_df.Age.dropna().mean()
train_df.Age  = train_df.Age.fillna(mean_age)
test_df.Age = test_df.Age.fillna(mean_age)

In [141]:
mean_fare = test_df.Fare.dropna().mean()
test_df.Fare = test_df.Fare.fillna(mean_fare)

In [142]:
train_df.columns


Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked'],
      dtype='object')

In [143]:
test_df.columns

Index(['PassengerId', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked'],
      dtype='object')

In [144]:
x_train = train_df[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']]
y_train = train_df['Survived']

x_test = test_df[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']]
idx = test_df['PassengerId']

x_train.shape,y_train.shape,  x_test.shape,idx.shape
                    

((891, 7), (891,), (418, 7), (418,))

In [145]:
#import warnings
#warnings.filterwarnings("ignore", category=FutureWarning)
logreg = LogisticRegression()
logreg.fit(x_train,y_train)

score_logreg = logreg.score(x_train,y_train)
print('logreg score is : ', score_logreg)
predict_logreg = logreg.predict(x_test)

logreg score is :  0.8047138047138047


In [146]:
rfc = RandomForestClassifier(n_estimators = 100)
rfc.fit(x_train,y_train)
score_rfc = rfc.score(x_train,y_train)
print('rfc score is :', score_rfc)
predict_rfc = rfc.predict(x_test)

rfc score is : 0.9820426487093153


In [147]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(x_train,y_train)
score_knn = knn.score(x_train,y_train)
print('knn sore is: ',score_knn)
predict_knn = knn.predict(x_test)

knn sore is:  0.8092031425364759


In [148]:
svc = SVC()
svc.fit(x_train,y_train)
score_svc = svc.score(x_train,y_train)
print('svc score is:', score_svc)
predict_svc = svc.predict(x_test)

svc score is: 0.8933782267115601


In [149]:
submission = pd.DataFrame({'PassengerId':idx,'Survived':predict_rfc})
submission.to_csv('newsub1.csv',index = False)